# TV Script Generation

## Data

In [1]:
import helper
import numpy as np
from collections import Counter
import problem_unittests as tests
import torch
import torch.nn as nn

In [2]:
data_dir = './data/Seinfeld_Scripts.txt'
text = helper.load_data(data_dir)

In [6]:
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

view_line_range = (0, 10)
lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 5.544240293684143

The lines 0 to 10:
jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! y

## Pre-processing
- Lookup Table
- Tokenize Punctuation

### Lookup Table
Create two dictionaries:
- Dictionary to go from the words to an id ->`vocab_to_int`
- Dictionary to go from the id to word ->`int_to_vocab`


In [7]:
def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """

    #Count frequency of words
    c = Counter(text)
    sorted_vocab = sorted(c, key=c.get, reverse=True)

    #create dictonaries that map integer values to words and the inverse
    vocab_to_int = {word: ii for ii, word in enumerate(sorted_vocab)}
    int_to_vocab = {ii: word for word, ii in vocab_to_int.items()}
    
    return (vocab_to_int,int_to_vocab)
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


In [8]:
sample_text = "this is some text to test something"
vocab_to_int, int_to_vocab = create_lookup_tables(sample_text)
print(vocab_to_int)
print(int_to_vocab)

{'t': 0, ' ': 1, 's': 2, 'e': 3, 'i': 4, 'o': 5, 'h': 6, 'm': 7, 'x': 8, 'n': 9, 'g': 10}
{0: 't', 1: ' ', 2: 's', 3: 'e', 4: 'i', 5: 'o', 6: 'h', 7: 'm', 8: 'x', 9: 'n', 10: 'g'}


### Tokenize Punctuation

In [9]:
def token_lookup():
    
    token_dict ={
        '.': '||Period||',
        ',': '||Comma||',
        '"': '||Quotation_Mark||',
        ';': '||Semicolon||',
        '!': '||Exclamation_mark||',
        '?': '||Question_mark||',
        '(': '||Left_Parentheses||',
        ')': '||Right_Parentheses||',
        '-': '||Dash||',
        "\n": '||Return||'
    }
    
    return token_dict
tests.test_tokenize(token_lookup)

Tests Passed


In [10]:
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point

In [2]:
int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

### Check Access to GPU 

In [3]:
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a Gpu to train your neural network')

No GPU found. Please use a Gpu to train your neural network


### Input

In [13]:
from torch.utils.data import TensorDataset, DataLoader

In [14]:
#batch the data
def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    #just full batches
    n_batches = len(words)//batch_size
    words = words[:n_batches*batch_size]
    
    #batch the data
    features = np.array([words[i:(i+sequence_length)] for i in range(len(words)-sequence_length)])
    targets = np.array([words[i+sequence_length] for i in range(len(words)-sequence_length)])
    
    #transform to tensor
    feature_tensor = torch.from_numpy(features)
    target_tensor = torch.from_numpy(targets)
    
    #define DataLoader
    data = TensorDataset(feature_tensor, target_tensor)
    data_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=False)
    
    return data_loader

In [15]:
# test dataloader

test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[  0,   1,   2,   3,   4],
        [  1,   2,   3,   4,   5],
        [  2,   3,   4,   5,   6],
        [  3,   4,   5,   6,   7],
        [  4,   5,   6,   7,   8],
        [  5,   6,   7,   8,   9],
        [  6,   7,   8,   9,  10],
        [  7,   8,   9,  10,  11],
        [  8,   9,  10,  11,  12],
        [  9,  10,  11,  12,  13]])

torch.Size([10])
tensor([  5,   6,   7,   8,   9,  10,  11,  12,  13,  14])


In [5]:
## Build the Neural Network

class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.3):
    
        super(RNN, self).__init__()
        
        self.vocab_size = vocab_size
        self.output_size = output_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.Embedding = nn.Embedding(vocab_size, embedding_dim)
        
        self.LSTM = nn.LSTM(embedding_dim, hidden_dim, n_layers,
                           dropout=dropout, batch_first=True)
        
        self.fc = nn.Linear(hidden_dim, output_size)
        
    def forward(self, nn_input, hidden):
        
        batch_size = nn_input.size(0)
        
        embeds = self.Embedding(nn_input)
        LSTM_out, hidden_out = self.LSTM(embeds, hidden)
        
        LSTM_out = LSTM_out.contiguous().view(-1, self.hidden_dim)
        
        out = self.fc(LSTM_out)
        
        # reshape into (batch_size, seq_length, output_size)
        out = out.view(batch_size, -1, self.output_size)
        # get last batch
        out = out[:, -1]
        
        return out, hidden
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        weight = next(self.parameters()).data
        
        if(train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                     weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                     weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
            
        
        return hidden

tests.test_rnn(RNN, train_on_gpu)

Tests Passed


## Define forward and backpropagation

In [19]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation on the neural network
    :param rnn: The PyTorch Module that holds the neural network
    :param optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    
    # TODO: Implement Function
    
    if(train_on_gpu):
        rnn.cuda()
    
    h = tuple([each.data for each in hidden])
    
    rnn.zero_grad()
    
    if(train_on_gpu):
        inp, target = inp.cuda(), target.cuda()
    
    output, h = rnn(inp, h)
    
    loss = criterion(output, target)
    loss.backward()
    
    nn.utils.clip_grad_norm_(rnn.parameters(), 5)
    
    optimizer.step()
    
    return loss.item(), h


tests.test_forward_back_prop(RNN, forward_back_prop, train_on_gpu)

Tests Passed


## Neural Network Training

In [20]:
def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []

    # returns a trained rnn
    return rnn

In [21]:
# Data params
# Sequence Length
sequence_length = 10
# Batch Size
batch_size = 128

# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

In [22]:
# Training parameters
# Number of Epochs
num_epochs = 10
# Learning Rate
learning_rate = 0.002

# Model parameters
# Vocab size
vocab_size = len(vocab_to_int)
# Output size
output_size = vocab_size
# Embedding Dimension
embedding_dim = 250
# Hidden Dimension
hidden_dim = 250
# Number of RNN Layers
n_layers = 2

# Show stats for every n number of batches
show_every_n_batches = 2000

### Train

In [23]:
# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.3)
if train_on_gpu:
    rnn.cuda()
print(rnn)

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr = learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

RNN(
  (Embedding): Embedding(21388, 250)
  (LSTM): LSTM(250, 250, num_layers=2, batch_first=True, dropout=0.3)
  (fc): Linear(in_features=250, out_features=21388, bias=True)
)
Training for 10 epoch(s)...
Epoch:    1/10    Loss: 4.994991242408752

Epoch:    1/10    Loss: 4.39775888299942

Epoch:    1/10    Loss: 4.264710321068764

Epoch:    2/10    Loss: 4.028546863569404

Epoch:    2/10    Loss: 3.8622880607843397

Epoch:    2/10    Loss: 3.8316683440208434

Epoch:    3/10    Loss: 3.73333630568255

Epoch:    3/10    Loss: 3.661201379060745

Epoch:    3/10    Loss: 3.625262855529785

Epoch:    4/10    Loss: 3.5675259732171853

Epoch:    4/10    Loss: 3.5156737643480303

Epoch:    4/10    Loss: 3.497494480252266

Epoch:    5/10    Loss: 3.4465078550850285

Epoch:    5/10    Loss: 3.408505469083786

Epoch:    5/10    Loss: 3.396619640469551

Epoch:    6/10    Loss: 3.353994098635054

Epoch:    6/10    Loss: 3.3346054565906527

Epoch:    6/10    Loss: 3.3198043427467345

Epoch:    7/10  

/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Model Trained and Saved


### Question: How did you decide on your model hyperparameters? 
For example, did you try different sequence_lengths and find that one size made the model converge faster? What about your hidden_dim and n_layers; how did you decide on those?

**Answer:** :  <br>
Configuration to achieve a Loss under 3.5: <br>
[lr = 0.001, batch_size = 128, dropout=0.5, seq_len=20, epochs=10, embedding_dim & hidden_dim = 250, Loss: 3.17] <br>
The loss were with 3.17 ok, but in the generated Scripts where strange mistakes. So I retrained with some modifications: <br>

Modify dropout from 0.5 t 0.3, decreace seq_len from 20 to 10, increase lr from 0.001 to 0.002. Those changes lead to a loss from 3.13, whihc is not uch different, but the generated sentences are much better.

## Checkpoint

In [6]:
_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

trained_rnn = torch.load('trained_rnn.pt', map_location='cpu')

## Generate Sequences

In [7]:
import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        # the generated word becomes the next "current sequence" and the cycle can continue
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences


In [12]:
# run the cell multiple times to get different results!
sequence_length = 10
gen_length = 400 # modify the length to your preference
prime_word = 'jerry' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

jerry: 28 jury, the contest, but i think we were in the mood.

george: i think it's a joke.

george: what did he say?

frank: no, it's a regular flight.

[new witness: joe bookman?

hoyt: so donald.

hoyt: yes, yes, it was.

hoyt: no, no, not a joke.

hoyt: yes, yes! i forgot the hardest ones who smothered me, and then they were innocentbystanders.

hoyt: no, i can't find it.

hoyt: yes, sure.

jerry: i dont have any quarters.

hoyt: what do i think, the jury?

jerry: what are you doing?

elaine: what?

soup nazi: yes!

[new witness: elaine's office]

[new witness: the most one of the robbery] are walking out the county court.

hoyt: so youre not going on the other parking lot of selfishness.

hoyt: no, not a lovely boy.

vandelay: yes!

frank: what are you talking about?

george: i had a good trip to the judge.

hoyt: so youre in a hurry.

hoyt: no, no, no. no.

jerry: no, that's not that bright restaurateur.

vandelay: objection overruled.

[new witness: elaine's bedroom]

hoyt: so d

In [14]:
#save generated text as .txt file

scriptname = "seinfield_script_2.txt"

with open(scriptname, "w") as file:
    file.write(generated_script)